### 本文的主要贡献
提供一个新的网络架构：纯attention(包括self attention 和 其他 attention)架构。<br>
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms to draw global dependencies between input and output, dispensing with recurrentce and convolutions entirely. The Transformer allows for significantly more parallelization and can be reach a new state of the art in translation quality.<br>
self attention: sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.
- RNN的限制： 

RNN网络的输出$h_t$同时受$h_{t-1}$和$input_t$的影响。This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples.<br>
这里的前后关系不是很明白，可以理解，他阻碍了并行，但为什么说这点在长句中是致命的？？

- CNN的限制

同样存在RNN的困扰，不过情节比较轻而已。
[22, 17, 9]all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet.
- attention的优势

可以很好的模拟长句内，不同部分（单词，短语）的依赖关系。<br>
The number of operations required to realte signals from two arbitrary input or output positions is reduced to a constant, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions.(multi-head attention)<br>
allowing modeling of dependencies without regard to their distance in the input or output sequences.<br>

### 模型架构
![Transformer](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/Transformer.png)

#### Encoder
由六个相同的子模型stack而成。每个子模型包括两层，即multi-head attention层和feed forward层（fc）。每个层都加入残缺机制（residual）并于其后进行bn操作。为了方便残缺求和，每层的输出的神经元个数等于输入embedding的长度，这里是512。（每层的输出为：LayerNorm(x + sublayer(x))）
#### Decoder
由六个相同的子模型stack而成。每个模型包括三层，即masked multi-head attention, multi-head attention层(这回部分输入由encoder获得)和feed forward层（fc）。应该注意的是，目标语言输入都向右平移一个位置。mask就是为了确保，当预测第i个输出的时候，他的信息只来自时间i之前的输出。通过对比输入句子和特殊字符的对比可以知道句子中是否存在特殊单词（像the之类的啊），并以此生成一个mask。后通过mask_fill_(mask, value)，把对应特殊单词的权重归零。<br>
this masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outpus at postions less than i.
#### Attention 
什么是attention。attention可以当成是一个映射方程，他把(query, key, value)作为输入，输出对应的output。其中query, key, value和output都是向量。 output是value的加权求和，权值由query和key决定。<br>
An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, key, values and output are all vectors. Thee output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.
##### Scaled Dot-Product Attention
![attention](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/Attention.png)
$Attention(Q,K,V)\ =\ softmax(\frac{QK^T}{\sqrt{d_k}})V$<br>
其中Q:[1, d], K:[n, d], V: [n, d], output: [1, d]<br>
d是代表每个单词的词向量的长度，n是输入的单词的数量。但query的数量m大于一的时候，Q的维度为对应的[m, d]，相应的结果输出维度为[m, d]<br>
引入scala因子的原因是：<br>
当使用点乘的时候，其值容易过大，导致导数偏低。使用scala因子可以抵消这种情况。<br>
We suspect that for large values of $d_k$, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients. To counteract this effect, we scale the dot products by $\frac{1}{\sqrt{d_k}}$. While for small values of $d_k$ the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for large value of $d_k$<br>
##### Multi-Head Attention
直接看公式比较容易懂：<br>
$MultiHead(Q, K, V) = Concat(head_1,...,\ head_h)W^O$<br>
$where\ head_1=Attention(QW_i^Q, KW_i^K, VW_i^V)$<br>
where the projections are parameter matrices$W_i^Q \in R^{d_{model}\ X\ d_k},\ W_i^K \in R^{d_{model}\ X\ d_k},\ W_i^V \in R^{R_{model}\ X\ d_v}$ and $W^O \in R^{hd_v\ X\ d_{model}}$<br>
输入一组三个矩阵V, K, Q，对三个矩阵分别进行h次映射，得到h组，每组三个矩阵。把各组矩阵分别扔进scaled Dot-Product Attention得到h组输出，串联起这些输出，最后把结果进行最后的映射，得到我们要的结果。<br>
V和K进行的是$d_{model}\ ->\ d_k$的映射。原来每个单词用$d_{model}$长度的向量表示，映射后每个单词用$d_k$长度的向量表示，其他不变。Q进行的映射与此类似。V和K映射后的单词维度应该相同是为了进行点乘。点乘的结果是对应各个单词的权重，也因此V映射后的单词维度可以与Q、K的不同。最后的一个映射是为了使最后输出的代表单词的词向量的长度能够回到原来的长度。<br>
一般的attention，就是对整个句子的加权求和，他没有办法同时关注多个地方，Multi Head Attention打破了这个限制。
Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averagin inhibits this.
##### 这些multi-head attention在这个模型中的应用
在这个模型中，对于multi-head attention主要由三种使用方法
- encoder-deocder attention

queries来自上一层decoder， keys和values来自encoder。
The queries come from the previous decoder layer, and the memory keys and values come from the output of encoder. This allows every position in the decoder to attend over all positions in the input sequence.
- encoder self-attention layer

queries, keys, values都来自上一层encoder层。
In self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer inthe encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder

- decoder self-attention layer

queries, keys, values都来自上一层decoder层。不同于encoder，一开始就获得整个句子的单词。decoder必须通过自回归一个个得到。所以第i个单词只能关注其之前的单词。我们通过mask实现这个功能。
self attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position. We need to prevent leftward information lfow in the decoder to preserve the auto-regressive property. We implement this inside of scaled dot-product attention by masking out(settign to -inf) all values in the input of the softmax which correspond to illegal connection.

#### Position-wise Feed-Forward Networks
$FFN(x)\ =\ max(0, xW_1+b_1)W_2+b+2$

#### Positional Encoding
因为没有使用RNN和CNN，所以无法获得位置相关的信息（绝对和相对位置）。为了突破这层阻碍，我们引入了Positional Encoding。和单词的embedding一样，我们训练一组向量来代表不同的position。为了方便与对应的单词进行结合（这里直接相加），我们使得位置向量的长度和单词向量长度相同。<br>
Positional Encoding的方法有很多种，这里用的是sine and cosine functions of different frequencies:
$PE_{pos, 2i} = sin (pos/10000^{2i/d_{model}})$<br>
$PE_{pos, 2i+1} = cos(pos/10000^{2i/d_{model}}$<br>
其中pos代表的是位置，i代表的是第i维。<br>
That is, each dimension of the positional encoding corresponds to a sinusoid. The wavelengths form a geometric progression from 2$\pi$ to 10000*2$\pi$.<br>
We chose this function because we hypothesized it would allow the model to easily learn to attend by relative positions, since for any fix offset k, $PE_{pos+k}$ can be represented as a linear function of $PE_{pos}$

### ？？？
RNN的限制中的as memory constraints limit batching across example不是很理解？？？<br>
Model Architecture中说的competitive neural sequence transduction models是指什么？？为什么这么称呼？？<br>
Model Architecture中的positional encoding具体不了解？？<br>
Model Architecture中的we also modify the self-attention sub-layer in the decoder stack to prevent positions from< attending to subsequent positions.不了解是怎么实现的，也不了解为什么要这么做。<br>
为什么decoder的输入都要向右位移一个位置。<br>
scaled Dot-product attention中有这么一句：In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. 问题是为什么在实践中，会同时搜索多个Query，一次不是指输出一个output吗？？

### 还要看的
[2] : additive attention
[4] : reading comprehension, self attention<br>
[9] : ConvS2S, CNN, learned positional embeddings<br>
[11]: residual connection
[17]: ByteNet, CNN<br>
[20]: fractorization tricks<br>
[21]: task-independent sentence representations, self attention<br>
[22]: extended Neural GPU, CNN<br>
[26]: abstractive summarization, self attention<br>
[27]: textual entailment, self attention<br>
[29]: share the same weight matrix between the two embedding layers, pre-sofmax linear transformation.<br>
[31]: conditional computation

##### masked_fill_(mask, value)
Fills elements of this tensor with value where mask is one. The shape of mask must be broadcastable with the shape of the underlying tensor.

##### numpy.tril(m, k=0)[source]

Lower triangle of an array.Return a copy of an array with elements above the k-th diagonal zeroed.<br>
Parameters:	<br>
m : array_like, shape (M, N) Input array.<br>
k : int, optional<br>
Diagonal above which to zero elements. k = 0 (the default) is the main diagonal, k < 0 is below it and k > 0 is above.
Returns:	<br>
tril : ndarray, shape (M, N)<br>
Lower triangle of m, of same shape and data-type as m.<br>